In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
### RUN IF IN COLAB

# # torch==1.10.2
# !pip install transformers==4.16.2 datasets==1.17.0 tokenizers==0.11.6 wandb==0.12.14

# !git clone -b kretov/small_addons https://github.com/DevKretov/ntu_nlp_al
# %cd ntu_nlp_al/

In [44]:
import torch
from transformers import get_scheduler
from datasets import list_metrics, load_metric

from active_learning_trainer import ALTrainer
from transformers import AutoTokenizer
from dataset import Dataset
from model import Model
from transformers import BertForSequenceClassification
from strategies import RandomStrategy

import wandb

In [36]:
parameters = dict()
parameters['use_gpu'] = True

parameters['weights_and_biases_on'] = False
parameters['weights_and_biases_key'] = '5e5e00356042a33b5cb271399b8d05c9c9d6ded8'
parameters['weights_and_biases_run_name'] = 'run_2'
# TODO: implement it
parameters['weights_and_biases_save_predictions'] = False

parameters['pretrained_model_name'] = 'prajjwal1/bert-tiny' #'distilbert-base-uncased'


# parameters['train_dataset_file_path'] = 'data/imdb/train_IMDB.csv'
# parameters['val_dataset_file_path'] = 'data/imdb/test_IMDB.csv'
# parameters['test_dataset_file_path'] = 'data/imdb/test_IMDB.csv'

parameters['train_dataset_file_path'] = 'data/news/train.csv'
parameters['val_dataset_file_path'] = 'data/news/val.csv'
parameters['test_dataset_file_path'] = 'data/news/test.csv'
parameters['dataset_file_delimiter'] = ','

parameters['dataset_text_column_name'] = 'text_cleaned' #'text'
parameters['dataset_label_column_name'] = 'label_reduced'#'airline_sentiment'

# TODO: implement this with CrossEntropyLoss
parameters['loss'] = 'cross_entropy'
parameters['loss_weighted'] = False

parameters['class_imbalance_reweight'] = True
parameters['train_batch_size'] = 32
parameters['val_batch_size'] = 64
parameters['test_batch_size'] = 64
parameters['epochs'] = 5
parameters['finetuned_model_type'] = 'classification'

parameters['al_iterations'] = 100
parameters['init_dataset_size'] = 32
parameters['add_dataset_size'] = 32
parameters['al_strategy'] = 'least_confidence' #'least_confidence'
parameters['full_train'] = False
parameters['debug'] = False

In [ ]:
if parameters['weights_and_biases_on']:
    wandb.login(key='5e5e00356042a33b5cb271399b8d05c9c9d6ded8')
    wandb.init(
        name=parameters['weights_and_biases_run_name'],
        project='ntu_al',
        reinit=True
    )

    wandb.config.update(parameters)

device = 'cpu'
if parameters['use_gpu']:
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(f'Device set to {device}!')

In [37]:
tokenizer = AutoTokenizer.from_pretrained(parameters['pretrained_model_name'])

dataset_obj = Dataset(tokenizer)

data_files = {
    'train': [parameters['train_dataset_file_path']],
    'val': [parameters['val_dataset_file_path']],
    'test': [parameters['test_dataset_file_path']]
}

dataset_obj.load_csv_dataset(
    data_files,
    delimiter=parameters['dataset_file_delimiter']
)

dataset_obj.truncate_dataset('train', 10000)
dataset_obj.truncate_dataset('val', 1000)
dataset_obj.truncate_dataset('test', 10000)

AL evaluation iteration. Batch   246/295:  83%|████████▎ | 246/295 [09:07<01:49,  2.23s/it]


Using custom data configuration default-e159e2f034d47ce8
Reusing dataset csv (/Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)
100%|██████████| 3/3 [00:00<00:00, 544.95it/s]


In [38]:
dataset_obj.prepare_labels(parameters['dataset_label_column_name'])
dataset_obj.encode_dataset(parameters['dataset_text_column_name'])

Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-ac32778fd5425486.arrow
Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-6799fe5e1a63afb8.arrow
Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-6853f82358e53875.arrow
Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-d7b79c90d7fbe0d7.arrow
Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d

In [39]:
print(f'Categories: {dataset_obj.get_all_categories()}')
num_labels = dataset_obj.get_num_categories()

Categories: {'alt': 0, 'comp': 1, 'misc': 2, 'rec': 3, 'sci': 4, 'soc': 5, 'talk': 6}


In [40]:
model = Model(
    parameters['pretrained_model_name'],
    model_type=parameters['finetuned_model_type'],
    num_labels=num_labels
)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [41]:
trainer = ALTrainer(
    wandb_on=parameters['weights_and_biases_on'],
    imbalanced_training=parameters['class_imbalance_reweight']
)
trainer.set_model(model)

# TODO: add strategy
trainer.set_strategy(None)
trainer.set_dataset(dataset_obj)
trainer.prepare_dataloaders(
    train_batch_size=parameters['train_batch_size'],
    val_batch_size=parameters['val_batch_size'],
    test_batch_size=parameters['test_batch_size'],
)

In [42]:
optimizer = torch.optim.AdamW(model.model.parameters(), lr=5e-5)
trainer.set_optimizer(optimizer)

num_training_steps = parameters['epochs'] * trainer.get_training_steps_num()

lr_scheduler = get_scheduler(
    name="linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
trainer.set_lr_scheduler(lr_scheduler)
trainer.set_device(device)



In [ ]:
trainer.add_evaluation_metric(load_metric('accuracy'))
trainer.add_evaluation_metric(load_metric('f1'))
trainer.add_evaluation_metric(load_metric('precision'))
trainer.add_evaluation_metric(load_metric('recall'))

In [43]:
if parameters['full_train']:
    trainer.full_train(
        train_epochs=parameters['epochs'],
        train_batch_size=parameters['train_batch_size'],
        val_batch_size=parameters['val_batch_size'],
        test_batch_size=parameters['test_batch_size'],
        debug=parameters['debug']
    )

trainer.al_train(
    al_iterations=parameters['al_iterations'],
    init_dataset_size=parameters['init_dataset_size'],
    add_dataset_size=parameters['add_dataset_size'],
    train_epochs=parameters['epochs'],
    strategy=parameters['al_strategy'],
    train_batch_size=parameters['train_batch_size'],
    val_batch_size=parameters['val_batch_size'],
    test_batch_size=parameters['test_batch_size'],
    debug=parameters['debug']
)

Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-9bd4ab3a27a7fb3a.arrow
Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-264ab8efe04d123a.arrow
Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-fd085d63684e4b7c.arrow
Loading cached processed dataset at /Users/antonkretov/.cache/huggingface/datasets/csv/default-e159e2f034d47ce8/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-078a83154302888e.arrow


Training initialized!
AL train dataset length: 32, rest dataset length: 9968
Training is run on 1 batches!
Evaluation is run on 16 batches!
Testing is run on 16 batches!


AL BADGE strategy applied!


AL iteration   1/100


Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

Model initialized.


Epoch   1
Training mean loss: 1.9942021369934082: 100%|██████████| 1/1 [00:00<00:00,  3.63it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 1.935079112648964: 100%|██████████| 16/16 [00:02<00:00,  7.25it/s]


Epoch   2
Training mean loss: 1.969994306564331: 100%|██████████| 1/1 [00:00<00:00,  4.01it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 1.9267708659172058: 100%|██████████| 16/16 [00:02<00:00,  7.46it/s]


Epoch   3
Training mean loss: 1.9879329204559326: 100%|██████████| 1/1 [00:00<00:00,  3.64it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 1.9196368902921677: 100%|██████████| 16/16 [00:02<00:00,  5.76it/s]


Epoch   4
Training mean loss: 1.9227190017700195: 100%|██████████| 1/1 [00:00<00:00,  3.30it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 1.9123196750879288: 100%|██████████| 16/16 [00:02<00:00,  6.70it/s]


Epoch   5
Training mean loss: 1.950924277305603: 100%

100%|██████████| 64/64 [00:00<00:00, 3129.24ex/s]


...


Updated AL datasets: train size = 3200, unlabelled size = 6800, sum: 10000 


AL iteration 100/100


Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

Model initialized.


Epoch   1
Training mean loss: 1.817127788066864: 100%|██████████| 100/100 [00:33<00:00,  2.94it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 1.6549407988786697: 100%|██████████| 16/16 [00:03<00:00,  4.95it/s]


Epoch   2
Training mean loss: 1.5098282980918885: 100%|██████████| 100/100 [00:33<00:00,  2.98it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 1.3942479118704796: 100%|██████████| 16/16 [00:03<00:00,  4.92it/s]


Epoch   3
Training mean loss: 1.2763945031166077: 100%|██████████| 100/100 [00:33<00:00,  3.00it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 1.1688376665115356: 100%|██████████| 16/16 [00:03<00:00,  5.03it/s]


Epoch   4
Training mean loss: 1.0502295452356338: 100%|██████████| 100/100 [00:34<00:00,  2.94it/s]

Epoch finished. Evaluation:
Running Eval mode...
Eval mean loss: 0.9836922995746136: 100%|██████████| 16/16 [00:03<00:00,  5.07it/s]


Epoch   5
Training mean loss: 0.8305

100%|██████████| 3232/3232 [00:01<00:00, 2819.40ex/s]



Updated AL datasets: train size = 3232, unlabelled size = 6768, sum: 10000 
